In [92]:
import boto3
import numpy as np
import pandas as pd
import psycopg2
import os
import yaml

In [93]:
# Load Redshift credentials.
with open('credentials.yml') as f:
    conf = yaml.load(f)
    
DBNAME = conf['login']['dbname']
HOST = conf['login']['host']
PORT = conf['login']['port']
USER = conf['login']['user']
PASSWORD = conf['login']['password']

In [86]:
# dc2.large
slices_per_node = 2
nodes = 2
slices = slices_per_node * nodes

In [12]:
# Database connection
con = psycopg2.connect(dbname=DBNAME, host=HOST, port=PORT, user=USER, password=PASSWORD)

In [ ]:
# Open a cursor to perform database operations
cur = con.cursor()

# Questions

* check the cluster configuration (nodes and slices)
* load data from parquet vs. csv vs. compressed data files (e.g. gzip)?
* load data copy vs. insert
* select operations defined sort key vs undefined sort key
* join operations defined distkey vs undefined distkey

## Load data

1. Create table
2. Create several files and upload to s3
3. Copy

TO CHECK:
* Manifest file to explicitly specify files (also guarantees consistency)
* Automatic Compression
* Optimizing Storage for Narrow Tables (due to hidden vars)
* update and insert new data -> and then vacuum
* UNLOAD data

## Create mock data

In [60]:
np.random.seed(123)
n_rows = 5*10**5
n_columns = 100
column_names = list(range(n_columns))
column_names = [str(c) for c in column_names]

df = pd.DataFrame(np.random.randint(0, 100, size=(n_rows, n_columns)), columns=column_names)

You can load table data from a single file, or you can split the data for each table into multiple files. The COPY command can load data from multiple files in parallel. You can load multiple files by specifying a common prefix, or prefix key, for the set, or by explicitly listing the files in a manifest file.

## Define table

## Load data

### to_csv

In [95]:
df.to_csv('data/single_numbers.csv', index=False, header=False)

### to_csv (compressed)

In [96]:
df.to_csv('data/single_numbers.gz', index=False, header=False, compression='gzip')

### to_parquet

In [97]:
df.to_parquet('data/single_numbers.parquet')

### to_csv (multiple files)
Amazon Redshift does not take file size into account when dividing the workload, so you need to ensure that the files are roughly the same size, between 1 MB and 1 GB after compression.

Split your data into files so that the number of files is a multiple of the number of slices in your cluster. That way Amazon Redshift can divide the data evenly among the slices. 

In [98]:
number_of_chunks = slices * 25
for chunk, df_i in enumerate(np.array_split(df, number_of_chunks)):
    path = 'data/split_numbers.csv.{}'.format(chunk)
    df_i.to_csv(path, index=False, header=False)

### to_parquet (multiple files)

In [99]:
for chunk, df_i in enumerate(np.array_split(df, number_of_chunks)):
    path = 'data/split_numbers.parquet.{}'.format(chunk)
    df_i.to_parquet(path)

## Upload to S3

In [100]:
AWS_ACCESS_KEY_ID = conf['aws_access']['key_id']
AWS_SECRET_ACCESS_KEY = conf['aws_access']['secret_key']
bucket_name = 'redshift-performance-data'

s3 = boto3.resource('s3', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
s3.create_bucket(Bucket=bucket_name)

s3.Bucket(name='redshift-performance-data')

In [103]:
for filename in os.listdir('data'):
        s3.Object(bucket_name, filename).put(Body=open(os.path.join('data', filename), 'rb'))